# Using an SAE as a steering vector

This notebook demonstrates how to use SAE lens to identify a feature on a pretrained model, and then construct a steering vector to affect the models output to various prompts. This notebook will also make use of Neuronpedia for identifying features of interest.

The steps below include:

- Installing relevant packages (Colab or locally)
- Load your SAE and the model it used
- Determining your feature of interest and its index
- Implementing your steering vector


## Setting up packages and notebook


### Import and installs


#### Environment Setup


In [ ]:

try:
    # for google colab users
    import google.colab  # type: ignore
    from google.colab import output

    COLAB = True
    %pip install sae-lens transformer-lens
except:
    # for local setup
    COLAB = False
    from IPython import get_ipython  # type: ignore

    ipython = get_ipython()
    assert ipython is not None
    ipython.run_line_magic("load_ext", "autoreload")
    ipython.run_line_magic("autoreload", "2")

# Imports for displaying vis in Colab / notebook
import webbrowser
import http.server
import socketserver
import threading

PORT = 8000

# general imports
import os
import torch
from tqdm import tqdm
import plotly.express as px

torch.set_grad_enabled(False);

In [ ]:
def display_vis_inline(filename: str, height: int = 850):
    """
    Displays the HTML files in Colab. Uses global `PORT` variable defined in prev cell, so that each
    vis has a unique port without having to define a port within the function.
    """
    if not (COLAB):
        webbrowser.open(filename)

    else:
        global PORT

        def serve(directory):
            os.chdir(directory)

            # Create a handler for serving files
            handler = http.server.SimpleHTTPRequestHandler

            # Create a socket server with the handler
            with socketserver.TCPServer(("", PORT), handler) as httpd:
                print(f"Serving files from {directory} on port {PORT}")
                httpd.serve_forever()

        thread = threading.Thread(target=serve, args=("/content",))
        thread.start()

        output.serve_kernel_port_as_iframe(
            PORT, path=f"/{filename}", height=height, cache_in_notebook=True
        )

        PORT += 1

#### General Installs and device setup


In [1]:
import os
os.environ['HF_HOME'] = '/mnt/datadisk/Mehdi'
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
import torch
# package import
from torch import Tensor
from transformer_lens import utils
from functools import partial
from jaxtyping import Int, Float

# device setup
if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {device}")

/home/mahdi/miniconda3/envs/myenv/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


Device: cuda


### Load your model and SAE

We're going to work with a pretrained GPT2-small model, and the RES-JB SAE set which is for the residual stream.


In [2]:
from transformer_lens import HookedTransformer
from sae_lens import SAE

# Choose a layer you want to focus on
# For this tutorial, we're going to use layer 2
layer = 12
# from sae_lens import SAE

release = "gemma-scope-2b-pt-res-canonical"
sae_id = "layer_25/width_16k/canonical"
sae = SAE.from_pretrained(release, sae_id, device=device)
# get model
model = HookedTransformer.from_pretrained("gemma-2-2b", device=device)

# # get the SAE for this layer
# sae = SAE.from_pretrained(
#     release="gemma-2b-res-jb", sae_id=f"blocks.{layer}.hook_resid_post", device=device
# )

# get hook point
hook_point = sae.cfg.metadata.hook_name
print(hook_point)

/home/mahdi/miniconda3/envs/myenv/lib/python3.10/site-packages/sae_lens/saes/sae.py:249: UserWarning: 
This SAE has non-empty model_from_pretrained_kwargs. 
For optimal performance, load the model like so:
model = HookedSAETransformer.from_pretrained_no_processing(..., **cfg.model_from_pretrained_kwargs)
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loaded pretrained model gemma-2-2b into HookedTransformer
blocks.25.hook_resid_post


## Determine your feature of interest and its index


### Find your feature


#### Explore through code by using the feature activations for a prompt

For the purpose of the tutorial, we are selecting a simple token prompt.

In this example we will look trying to find and steer a "Jedi" feature.

We run our prompt on our model and get the cache, which we then use with our sae to get our feature activations.

Now we'll look at the top feature activations and look them up on Neuronpedia to determine what they have been intepreted as.


In [3]:
sv_prompt = "London"
sv_logits, cache = model.run_with_cache(sv_prompt, prepend_bos=True)
tokens = model.to_tokens(sv_prompt)
print(tokens)

# get the feature activations from our SAE
sv_feature_acts = sae.encode(cache[hook_point])

# get sae_out
sae_out = sae.decode(sv_feature_acts)

# print out the top activations, focus on the indices
print(torch.topk(sv_feature_acts, 10))

tensor([[    2, 19748]], device='cuda:0')
torch.return_types.topk(
values=tensor([[[2302.4790,  919.2746,  913.3135,  362.6923,  330.1858,  330.0836,
           225.1641,  187.3522,  162.5361,  161.0491],
         [ 168.2361,  149.9995,   87.2618,   83.3183,   73.4003,   67.7044,
            59.5774,   42.6970,   42.0955,   40.7080]]], device='cuda:0',
       grad_fn=<TopkBackward0>),
indices=tensor([[[ 7479,  5149, 14186, 14325, 15861, 15298,  1734,  6461,   484,  5911],
         [14325,  1510,  3017,  8657,  8356,  1076, 16369, 15920, 12871, 15275]]],
       device='cuda:0'))


In [5]:
from sae_lens.analysis.neuronpedia_integration import get_neuronpedia_quick_list

get_neuronpedia_quick_list(
    sae=sae,
    features=torch.topk(sv_feature_acts, 3).indices.tolist(),
)

'https://neuronpedia.org/quick-list/?name=temporary_list&features=%5B%7B%22modelId%22%3A%20%22gemma-2b%22%2C%20%22layer%22%3A%20%226-res-jb%22%2C%20%22index%22%3A%20%22%5B%5B3390%2C%2015881%2C%205347%5D%2C%20%5B6518%2C%2013743%2C%201959%5D%2C%20%5B1571%2C%2012529%2C%2015173%5D%2C%20%5B12773%2C%2010200%2C%2015173%5D%2C%20%5B5192%2C%2015173%2C%2012030%5D%5D%22%7D%5D'

As we can see from our print out of tokens, the prompt is made of three tokens in total - "<endoftext>", "J", and "edi".

Our feature activation indexes at sv_feature_acts[2] - for "edi" - are of most interest to us.

Because we are using pretrained saes that have published feature maps, you can search on Neuronpedia for a feature of interest.


### Steps for Neuronpedia use

Use the interface to search for a specific concept or item and determine which layer and at what index it is.

1.  Open the [Neuronpedia](https://www.neuronpedia.org/) homepage.
2.  Using the "Models" dropdown, select your model. Here we are using GPT2-SM (GPT2-small).
3.  The next page will have a search bar, which allows you to enter your index of interest. We're interested in the "RES-JB" SAE set, make sure to select it.
4.  We found these indices in the previous step: [ 7650, 718, 22372]. Select them in the search to see the feature dashboard for each.
5.  As we'll see, some of the indices may relate to features you don't care about.

From using Neuronpedia, I have determined that my feature of interest is in layer 2, at index 7650: [here](https://www.neuronpedia.org/gpt2-small/2-res-jb/7650) is the feature.


### Note: 2nd Option - Starting with Neuronpedia

Another option here is that you can start with Neuronpedia to identify features of interest. By using your prompt in the interface you can explore which features were involved and search across all the layers. This allows you to first determine your layer and index of interest in Neuronpedia before focusing them in your code. Start [here](https://www.neuronpedia.org/search) if you want to begin with search.


## Implement your steering vector and affect the output


### Define values for your steering vector

To create our steering vector, we now need to get the decoder weights from our sparse autoencoder found at our index of interest.

Then to use our steering vector, we want a prompt for text generation, as well as a scaling factor coefficent to apply with the steering vector

We also set common sampling kwargs - temperature, top_p and freq_penalty


In [4]:
steering_vector = sae.W_dec[3026]

example_prompt = "What is the most iconic structure known to man?"
coeff = 300
sampling_kwargs = dict(temperature=1.0, top_p=0.1, freq_penalty=1.0)

### Set up hook functions

Finally, we need to create a hook that allows us to apply the steering vector when our model runs generate() on our defined prompt. We have also added a boolean value 'steering_on' that allows us to easily toggle the steering vector on and off for each prompt


In [5]:
def steering_hook(resid_pre, hook):
    if resid_pre.shape[1] == 1:
        return

    position = sae_out.shape[1]
    if steering_on:
        # using our steering vector and applying the coefficient
        resid_pre[:, : position - 1, :] += coeff * steering_vector


def hooked_generate(prompt_batch, fwd_hooks=[], seed=None, **kwargs):
    if seed is not None:
        torch.manual_seed(seed)

    with model.hooks(fwd_hooks=fwd_hooks):
        tokenized = model.to_tokens(prompt_batch)
        result = model.generate(
            stop_at_eos=False,  # avoids a bug on MPS
            input=tokenized,
            max_new_tokens=50,
            do_sample=True,
            **kwargs,
        )
    return result

In [6]:
layer

12

In [7]:
def run_generate(example_prompt):
    model.reset_hooks()
    editing_hooks = [(f"blocks.{20}.hook_resid_post", steering_hook)]
    res = hooked_generate(
        [example_prompt] * 3, editing_hooks, seed=None, **sampling_kwargs
    )

    # Print results, removing the ugly beginning of sequence token
    res_str = model.to_string(res[:, 1:])
    print(("\n\n" + "-" * 80 + "\n\n").join(res_str))

### Generate text influenced by steering vector

You may want to experiment with the scaling factor coefficient value that you set and see how it affects the generated output.


In [8]:
steering_on = True
run_generate(example_prompt)

  0%|          | 0/50 [00:00<?, ?it/s]

What is the most iconic structure known to man? The Eiffel Tower, the Great Wall of China, or maybe even the Statue of Liberty?

Well, if you’re a fan of <em>The Simpsons</em>, then you know that it’s actually a giant statue of Homer Simpson.



--------------------------------------------------------------------------------

What is the most iconic structure known to man? The Eiffel Tower, the Great Wall of China, or maybe even the Statue of Liberty?

Well, if you’re a fan of <em>The Simpsons</em>, then you know that it’s actually a giant donut.

That’s

--------------------------------------------------------------------------------

What is the most iconic structure known to man? The Eiffel Tower, the Great Wall of China, or maybe even the Statue of Liberty?

If you’re a fan of <em>The Simpsons</em>, then you know that it’s none other than Springfield’s own Krusty Burger.


### Generate text with no steering


In [9]:
steering_on = False
run_generate(example_prompt)

  0%|          | 0/50 [00:00<?, ?it/s]

What is the most iconic structure known to man? The answer is the Great Pyramid of Giza. It was built by Pharaoh Khufu, who ruled Egypt from 2589 BC to 2566 BC.

The pyramid was built as a tomb for Khufu and his family

--------------------------------------------------------------------------------

What is the most iconic structure known to man? The answer is the Great Pyramid of Giza. It was built by Pharaoh Khufu, who ruled Egypt from 2589 BC to 2566 BC.

The pyramid was built as a tomb for Khufu and his family

--------------------------------------------------------------------------------

What is the most iconic structure known to man? The answer is the Great Pyramid of Giza. It was built by Pharaoh Khufu, who ruled Egypt from 2589 BC to 2566 BC.

The pyramid was built as a tomb for Khufu and his family


### General Question test

We'll also attempt a more general prompt which is a better indication of whether our steering vector is having an effect or not


In [10]:
question_prompt = "What is on your mind?"
coeff = 100
sampling_kwargs = dict(temperature=1.0, top_p=0.1, freq_penalty=1.0)

In [11]:
steering_on = True
run_generate(question_prompt)

  0%|          | 0/50 [00:00<?, ?it/s]

What is on your mind?

[User 0001]

I'm thinking about the fact that I have a lot of work to do.
 

[User 0002]

I'm thinking about how much I hate my job

--------------------------------------------------------------------------------

What is on your mind?

[User 0001]

I'm thinking about the fact that I have a lot of work to do.
 

[User 0002]

I'm thinking about how much I hate my job

--------------------------------------------------------------------------------

What is on your mind?

[User 0001]

I'm thinking about the fact that I have a lot of work to do.
 

[User 0002]

I'm thinking about how much I hate my job


In [40]:
steering_on = False
run_generate(question_prompt)

  0%|          | 0/50 [00:00<?, ?it/s]

What is on your mind?

[User 0001]

I'm thinking about the fact that I have a lot of work to do.
 

[User 0002]

I'm thinking about how much I hate my job

--------------------------------------------------------------------------------

What is on your mind?

[User 0001]

I'm thinking about the fact that I have a lot of work to do.
 

[User 0002]

I'm thinking about how much I hate my job

--------------------------------------------------------------------------------

What is on your mind?

[User 0001]

I'm thinking about the fact that I have a lot of work to do.
 

[User 0002]

I'm thinking about how much I hate my job


## Next Steps

Ideas you could take for further exploration:

- Try ablating the feature
- Try and get a response where just the feature token prints over and over
- Investigate other features with more complex usage
